In [ ]:
# 需要装两个依赖
#!pip install gensim
#!pip install hanziconv

# Step1 将wiki xml转化为txt

In [23]:
# -*- coding: utf-8 -*-
import logging
from gensim.corpora import WikiCorpus


class Wiki2Text:
    
    def __init__(self):
        # 定义日志输出
        logger = logging.getLogger()
        logger.handlers.clear()  # 清除所有已存在的 handlers
        logging.basicConfig(format = '%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
    
    def wiki_to_text(self, wiki_path: str):
        logging.info(f"开始下载wiki文件:{wiki_path}")
        
        # 创建corpus对象
        wiki_corpus = WikiCorpus(wiki_path, dictionary={})
        
        # 将xml转化为txt
        with open("wiki_text.txt", 'w', encoding='utf-8') as output:
            text_count = 0
            for text in wiki_corpus.get_texts():
                # 这里的text是文档里句子的list集合
                output.write(' '.join(text) + '\n')
                text_count += 1
        
                if not text_count % 10000:
                    logging.info(f"目前已经处理{text_count}篇wiki文档")
            
            logging.info(f"wiki文档处理完毕！")
            

if __name__ == "__main__":
    wiki = Wiki2Text()
    wiki.wiki_to_text("zhwiki-20240820-pages-articles-multistream.xml.bz2")
    



2024-08-30 23:36:33,574 : INFO : 开始下载wiki文件:zhwiki-20240820-pages-articles-multistream.xml.bz2
2024-08-30 23:37:03,032 : INFO : 目前已经处理10000篇wiki文档
2024-08-30 23:37:27,396 : INFO : 目前已经处理20000篇wiki文档
2024-08-30 23:37:49,229 : INFO : 目前已经处理30000篇wiki文档
2024-08-30 23:38:09,868 : INFO : 目前已经处理40000篇wiki文档
2024-08-30 23:38:30,141 : INFO : 目前已经处理50000篇wiki文档
2024-08-30 23:38:50,960 : INFO : 目前已经处理60000篇wiki文档
2024-08-30 23:39:10,119 : INFO : 目前已经处理70000篇wiki文档
2024-08-30 23:39:29,283 : INFO : 目前已经处理80000篇wiki文档
2024-08-30 23:39:47,555 : INFO : 目前已经处理90000篇wiki文档
2024-08-30 23:40:05,418 : INFO : 目前已经处理100000篇wiki文档
2024-08-30 23:40:25,790 : INFO : 目前已经处理110000篇wiki文档
2024-08-30 23:40:50,082 : INFO : 目前已经处理120000篇wiki文档
2024-08-30 23:41:09,945 : INFO : 目前已经处理130000篇wiki文档
2024-08-30 23:41:32,233 : INFO : 目前已经处理140000篇wiki文档
2024-08-30 23:41:52,993 : INFO : 目前已经处理150000篇wiki文档
2024-08-30 23:42:14,039 : INFO : 目前已经处理160000篇wiki文档
2024-08-30 23:42:35,197 : INFO : 目前已经处理170000篇wiki文档
2024-08-30 23

## Step2 将繁体转换为简体，并去除去用词

In [29]:
# -*- coding:utf-8 -*-
import jieba
import logging
from hanziconv import HanziConv

class Segmentation:
    
    def __init__(self):
        logging.basicConfig(format = '%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
        self.sw = set()
        
    # 加载stopwords
    def set_stopwords(self):
        with open('stopwords.txt') as stopwords:
            for sw in stopwords:
                self.sw.add(sw.strip('\n'))
        logging.info('stopwords加载完毕')
        
    # 将繁体->简体
    def traditional_2_simplified(self):
        logging.info('开始将繁体字转化为简体字')
        
        simplified = open('simplified.txt', 'w', encoding='utf-8')
        
        with open('wiki_text.txt', 'r', encoding='utf-8') as traditional:
            for s in traditional:
                simplified.write(HanziConv.toSimplified(s))
        
        simplified.close()
        logging.info('繁体->简体转化成功')
        
    
    # 去除停用词
    def segmentation(self):
        logging.info('开始去除停用词')
        
        segmentation = open('segmentation.txt', 'w', encoding = 'utf-8')
        with open('simplified.txt', 'r', encoding='utf-8') as simplified:
            for s in simplified:
                seg_words = jieba.cut(s)
                rm_stop_words = [w for w in seg_words if w not in self.sw]
                
                segmentation.write(' '.join(rm_stop_words))
            
        segmentation.close()
        logging.info('停用词去除完毕！')

if __name__ == '__main__':
    segmentation = Segmentation()
    
    segmentation.set_stopwords()
    
    segmentation.traditional_2_simplified()
    
    segmentation.segmentation()
            

2024-08-31 00:45:47,334 : INFO : stopwords加载完毕
2024-08-31 00:45:47,336 : INFO : 开始将繁体字转化为简体字
2024-08-31 00:53:29,690 : INFO : 繁体->简体转化成功
2024-08-31 00:53:29,692 : INFO : 开始去除停用词
Building prefix dict from the default dictionary ...
2024-08-31 00:53:29,747 : DEBUG : Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/cn/wgmtytt14wd_2h240_w7088c0000gn/T/jieba.cache
2024-08-31 00:53:29,750 : DEBUG : Loading model from cache /var/folders/cn/wgmtytt14wd_2h240_w7088c0000gn/T/jieba.cache
Loading model cost 0.480 seconds.
2024-08-31 00:53:30,229 : DEBUG : Loading model cost 0.480 seconds.
Prefix dict has been built successfully.
2024-08-31 00:53:30,231 : DEBUG : Prefix dict has been built successfully.
2024-08-31 01:24:24,070 : INFO : 停用词去除完毕！


## Step 3 开始使用Gension Word2Vec进行训练

In [30]:
# -*- coding: utf-9 -*-
from gensim.models import word2vec

class Train:
    
    def __init__(self):
        pass
    
    def train(self):
        logging.info("开始训练word2vec模型")
        
        # 预处理文本
        sentence = word2vec.Text8Corpus("segmentation.txt")
        # 设置模型训练参数
        # window上下文窗口：在训练时，模型会尝试预测目标单词前后 window 范围内的单词。window=10 表示模型考虑目标词前后 10 个单词作为上下文。
        # min_count: 忽略在整个语料库中出现次数低于 min_count 的单词。如果一个单词在整个语料库中出现的次数少于 5 次，它将被忽略，不参与训练。这是为了减少模型对稀有词汇的噪声学习，并减少模型的大小。
        # worker: 指定使用多少个线程进行训练。这是为了并行化训练过程，以加快模型的训练速度。
        # sg: sg=1 表示使用 Skip-gram 模型，而 sg=0 表示使用 CBOW（连续袋模型）模型
        model = word2vec.Word2Vec(sentence, window=10, min_count=5, workers=4, sg=1)
        # 保存模型
        model.wv.save_word2vec_format(u'wiki300.model.bin', binary=True)
        
        logging.info("开始训练word2vec模型")
        
if __name__ == '__main__':
    t = Train()
    t.train()

2024-08-31 01:29:42,543 : INFO : 开始训练word2vec模型
2024-08-31 01:29:42,556 : INFO : collecting all words and their counts
2024-08-31 01:29:42,560 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2024-08-31 01:30:10,783 : INFO : PROGRESS: at sentence #10000, processed 100000000 words, keeping 2476045 word types
2024-08-31 01:30:40,351 : INFO : PROGRESS: at sentence #20000, processed 200000000 words, keeping 3785913 word types
2024-08-31 01:30:56,513 : INFO : collected 4400491 word types from a corpus of 256349648 raw words and 25635 sentences
2024-08-31 01:30:56,513 : INFO : Creating a fresh vocabulary
2024-08-31 01:31:00,523 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=5 retains 1035863 unique words (23.54% of original 4400491, drops 3364628)', 'datetime': '2024-08-31T01:30:59.926765', 'gensim': '4.3.3', 'python': '3.8.5 (default, Sep  4 2020, 02:22:02) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'prepare_vocab'}
20

2024-08-31 01:32:06,763 : INFO : EPOCH 0 - PROGRESS: at 3.75% examples, 174105 words/s, in_qsize 7, out_qsize 0
2024-08-31 01:32:07,777 : INFO : EPOCH 0 - PROGRESS: at 3.82% examples, 174065 words/s, in_qsize 7, out_qsize 0
2024-08-31 01:32:08,808 : INFO : EPOCH 0 - PROGRESS: at 3.89% examples, 173987 words/s, in_qsize 8, out_qsize 0
2024-08-31 01:32:09,816 : INFO : EPOCH 0 - PROGRESS: at 3.94% examples, 172953 words/s, in_qsize 7, out_qsize 0
2024-08-31 01:32:10,916 : INFO : EPOCH 0 - PROGRESS: at 4.01% examples, 172511 words/s, in_qsize 7, out_qsize 0
2024-08-31 01:32:11,928 : INFO : EPOCH 0 - PROGRESS: at 4.06% examples, 171691 words/s, in_qsize 7, out_qsize 0
2024-08-31 01:32:12,931 : INFO : EPOCH 0 - PROGRESS: at 4.13% examples, 171724 words/s, in_qsize 7, out_qsize 0
2024-08-31 01:32:14,026 : INFO : EPOCH 0 - PROGRESS: at 4.20% examples, 171665 words/s, in_qsize 7, out_qsize 0
2024-08-31 01:32:15,028 : INFO : EPOCH 0 - PROGRESS: at 4.28% examples, 171853 words/s, in_qsize 7, out_

2024-08-31 01:33:24,933 : INFO : EPOCH 0 - PROGRESS: at 9.24% examples, 174136 words/s, in_qsize 8, out_qsize 0
2024-08-31 01:33:25,966 : INFO : EPOCH 0 - PROGRESS: at 9.31% examples, 174169 words/s, in_qsize 7, out_qsize 0
2024-08-31 01:33:26,976 : INFO : EPOCH 0 - PROGRESS: at 9.39% examples, 174221 words/s, in_qsize 7, out_qsize 0
2024-08-31 01:33:27,993 : INFO : EPOCH 0 - PROGRESS: at 9.46% examples, 174277 words/s, in_qsize 7, out_qsize 0
2024-08-31 01:33:28,995 : INFO : EPOCH 0 - PROGRESS: at 9.53% examples, 174277 words/s, in_qsize 7, out_qsize 0
2024-08-31 01:33:29,999 : INFO : EPOCH 0 - PROGRESS: at 9.60% examples, 174340 words/s, in_qsize 7, out_qsize 0
2024-08-31 01:33:31,015 : INFO : EPOCH 0 - PROGRESS: at 9.68% examples, 174390 words/s, in_qsize 7, out_qsize 0
2024-08-31 01:33:32,041 : INFO : EPOCH 0 - PROGRESS: at 9.75% examples, 174423 words/s, in_qsize 7, out_qsize 0
2024-08-31 01:33:33,092 : INFO : EPOCH 0 - PROGRESS: at 9.83% examples, 174430 words/s, in_qsize 7, out_

2024-08-31 01:34:40,762 : INFO : EPOCH 0 - PROGRESS: at 14.64% examples, 175132 words/s, in_qsize 7, out_qsize 0
2024-08-31 01:34:41,823 : INFO : EPOCH 0 - PROGRESS: at 14.72% examples, 175171 words/s, in_qsize 8, out_qsize 0
2024-08-31 01:34:42,895 : INFO : EPOCH 0 - PROGRESS: at 14.80% examples, 175200 words/s, in_qsize 7, out_qsize 0
2024-08-31 01:34:43,970 : INFO : EPOCH 0 - PROGRESS: at 14.87% examples, 175226 words/s, in_qsize 7, out_qsize 0
2024-08-31 01:34:45,047 : INFO : EPOCH 0 - PROGRESS: at 14.95% examples, 175252 words/s, in_qsize 7, out_qsize 0
2024-08-31 01:34:46,116 : INFO : EPOCH 0 - PROGRESS: at 15.03% examples, 175278 words/s, in_qsize 7, out_qsize 0
2024-08-31 01:34:47,196 : INFO : EPOCH 0 - PROGRESS: at 15.11% examples, 175297 words/s, in_qsize 7, out_qsize 0
2024-08-31 01:34:48,273 : INFO : EPOCH 0 - PROGRESS: at 15.19% examples, 175322 words/s, in_qsize 7, out_qsize 0
2024-08-31 01:34:49,340 : INFO : EPOCH 0 - PROGRESS: at 15.26% examples, 175352 words/s, in_qsiz

2024-08-31 01:35:57,914 : INFO : EPOCH 0 - PROGRESS: at 20.06% examples, 174972 words/s, in_qsize 7, out_qsize 0
2024-08-31 01:35:58,950 : INFO : EPOCH 0 - PROGRESS: at 20.14% examples, 174982 words/s, in_qsize 7, out_qsize 0
2024-08-31 01:35:59,983 : INFO : EPOCH 0 - PROGRESS: at 20.21% examples, 174961 words/s, in_qsize 7, out_qsize 0
2024-08-31 01:36:01,008 : INFO : EPOCH 0 - PROGRESS: at 20.28% examples, 174976 words/s, in_qsize 7, out_qsize 0
2024-08-31 01:36:02,092 : INFO : EPOCH 0 - PROGRESS: at 20.36% examples, 174991 words/s, in_qsize 7, out_qsize 0
2024-08-31 01:36:03,189 : INFO : EPOCH 0 - PROGRESS: at 20.44% examples, 174997 words/s, in_qsize 7, out_qsize 0
2024-08-31 01:36:04,235 : INFO : EPOCH 0 - PROGRESS: at 20.51% examples, 174999 words/s, in_qsize 7, out_qsize 0
2024-08-31 01:36:05,259 : INFO : EPOCH 0 - PROGRESS: at 20.58% examples, 174983 words/s, in_qsize 7, out_qsize 0
2024-08-31 01:36:06,290 : INFO : EPOCH 0 - PROGRESS: at 20.66% examples, 174994 words/s, in_qsiz

...

2024-08-31 10:06:10,711 : INFO : EPOCH 4 - PROGRESS: at 74.91% examples, 177885 words/s, in_qsize 7, out_qsize 0
2024-08-31 10:06:11,791 : INFO : EPOCH 4 - PROGRESS: at 74.98% examples, 177887 words/s, in_qsize 7, out_qsize 0
2024-08-31 10:06:12,868 : INFO : EPOCH 4 - PROGRESS: at 75.06% examples, 177890 words/s, in_qsize 7, out_qsize 0
2024-08-31 10:06:13,935 : INFO : EPOCH 4 - PROGRESS: at 75.14% examples, 177893 words/s, in_qsize 8, out_qsize 0
2024-08-31 10:06:15,013 : INFO : EPOCH 4 - PROGRESS: at 75.22% examples, 177895 words/s, in_qsize 7, out_qsize 0
2024-08-31 10:06:16,081 : INFO : EPOCH 4 - PROGRESS: at 75.30% examples, 177898 words/s, in_qsize 7, out_qsize 0
2024-08-31 10:06:17,163 : INFO : EPOCH 4 - PROGRESS: at 75.37% examples, 177900 words/s, in_qsize 7, out_qsize 0
2024-08-31 10:06:18,228 : INFO : EPOCH 4 - PROGRESS: at 75.45% examples, 177904 words/s, in_qsize 7, out_qsize 0
2024-08-31 10:06:19,268 : INFO : EPOCH 4 - PROGRESS: at 75.53% examples, 177903 words/s, in_qsiz

2024-08-31 10:07:27,380 : INFO : EPOCH 4 - PROGRESS: at 80.40% examples, 177913 words/s, in_qsize 7, out_qsize 0
2024-08-31 10:07:28,462 : INFO : EPOCH 4 - PROGRESS: at 80.48% examples, 177914 words/s, in_qsize 7, out_qsize 0
2024-08-31 10:07:29,548 : INFO : EPOCH 4 - PROGRESS: at 80.56% examples, 177915 words/s, in_qsize 7, out_qsize 0
2024-08-31 10:07:30,625 : INFO : EPOCH 4 - PROGRESS: at 80.64% examples, 177917 words/s, in_qsize 7, out_qsize 0
2024-08-31 10:07:31,634 : INFO : EPOCH 4 - PROGRESS: at 80.70% examples, 177904 words/s, in_qsize 7, out_qsize 0
2024-08-31 10:07:32,648 : INFO : EPOCH 4 - PROGRESS: at 80.78% examples, 177908 words/s, in_qsize 7, out_qsize 0
2024-08-31 10:07:33,739 : INFO : EPOCH 4 - PROGRESS: at 80.85% examples, 177908 words/s, in_qsize 7, out_qsize 0
2024-08-31 10:07:34,833 : INFO : EPOCH 4 - PROGRESS: at 80.92% examples, 177891 words/s, in_qsize 6, out_qsize 1
2024-08-31 10:07:35,841 : INFO : EPOCH 4 - PROGRESS: at 80.99% examples, 177885 words/s, in_qsiz

2024-08-31 10:08:44,742 : INFO : EPOCH 4 - PROGRESS: at 85.88% examples, 177800 words/s, in_qsize 7, out_qsize 0
2024-08-31 10:08:45,828 : INFO : EPOCH 4 - PROGRESS: at 85.96% examples, 177801 words/s, in_qsize 7, out_qsize 0
2024-08-31 10:08:46,925 : INFO : EPOCH 4 - PROGRESS: at 86.03% examples, 177801 words/s, in_qsize 7, out_qsize 0
2024-08-31 10:08:48,013 : INFO : EPOCH 4 - PROGRESS: at 86.11% examples, 177801 words/s, in_qsize 7, out_qsize 0
2024-08-31 10:08:49,024 : INFO : EPOCH 4 - PROGRESS: at 86.18% examples, 177789 words/s, in_qsize 7, out_qsize 0
2024-08-31 10:08:50,031 : INFO : EPOCH 4 - PROGRESS: at 86.25% examples, 177794 words/s, in_qsize 7, out_qsize 0
2024-08-31 10:08:51,121 : INFO : EPOCH 4 - PROGRESS: at 86.33% examples, 177794 words/s, in_qsize 7, out_qsize 0
2024-08-31 10:08:52,202 : INFO : EPOCH 4 - PROGRESS: at 86.41% examples, 177796 words/s, in_qsize 7, out_qsize 0
2024-08-31 10:08:53,290 : INFO : EPOCH 4 - PROGRESS: at 86.49% examples, 177796 words/s, in_qsiz

2024-08-31 10:10:02,781 : INFO : EPOCH 4 - PROGRESS: at 91.47% examples, 177804 words/s, in_qsize 8, out_qsize 0
2024-08-31 10:10:03,853 : INFO : EPOCH 4 - PROGRESS: at 91.55% examples, 177806 words/s, in_qsize 7, out_qsize 0
2024-08-31 10:10:04,935 : INFO : EPOCH 4 - PROGRESS: at 91.62% examples, 177808 words/s, in_qsize 7, out_qsize 0
2024-08-31 10:10:06,025 : INFO : EPOCH 4 - PROGRESS: at 91.70% examples, 177808 words/s, in_qsize 7, out_qsize 0
2024-08-31 10:10:07,100 : INFO : EPOCH 4 - PROGRESS: at 91.78% examples, 177809 words/s, in_qsize 7, out_qsize 0
2024-08-31 10:10:08,174 : INFO : EPOCH 4 - PROGRESS: at 91.86% examples, 177812 words/s, in_qsize 7, out_qsize 0
2024-08-31 10:10:09,259 : INFO : EPOCH 4 - PROGRESS: at 91.94% examples, 177812 words/s, in_qsize 7, out_qsize 0
2024-08-31 10:10:10,340 : INFO : EPOCH 4 - PROGRESS: at 92.01% examples, 177813 words/s, in_qsize 7, out_qsize 0
2024-08-31 10:10:11,398 : INFO : EPOCH 4 - PROGRESS: at 92.09% examples, 177817 words/s, in_qsiz

2024-08-31 10:11:20,434 : INFO : EPOCH 4 - PROGRESS: at 97.05% examples, 177844 words/s, in_qsize 8, out_qsize 0
2024-08-31 10:11:21,515 : INFO : EPOCH 4 - PROGRESS: at 97.13% examples, 177845 words/s, in_qsize 8, out_qsize 0
2024-08-31 10:11:22,594 : INFO : EPOCH 4 - PROGRESS: at 97.20% examples, 177832 words/s, in_qsize 8, out_qsize 0
2024-08-31 10:11:23,673 : INFO : EPOCH 4 - PROGRESS: at 97.27% examples, 177834 words/s, in_qsize 7, out_qsize 0
2024-08-31 10:11:24,752 : INFO : EPOCH 4 - PROGRESS: at 97.35% examples, 177836 words/s, in_qsize 7, out_qsize 0
2024-08-31 10:11:25,832 : INFO : EPOCH 4 - PROGRESS: at 97.43% examples, 177837 words/s, in_qsize 7, out_qsize 0
2024-08-31 10:11:26,918 : INFO : EPOCH 4 - PROGRESS: at 97.51% examples, 177838 words/s, in_qsize 7, out_qsize 0
2024-08-31 10:11:27,989 : INFO : EPOCH 4 - PROGRESS: at 97.59% examples, 177841 words/s, in_qsize 7, out_qsize 0
2024-08-31 10:11:29,056 : INFO : EPOCH 4 - PROGRESS: at 97.66% examples, 177842 words/s, in_qsiz

In [36]:
from gensim.models.keyedvectors import KeyedVectors

if __name__ == '__main__':
    word_vectors = KeyedVectors.load_word2vec_format("wiki300.model.bin", binary=True)
    
    res = word_vectors.most_similar("坚强", topn = 10)
    
    for item in res:
        print(item[0], item[1])
    
    print("坚强、坚韧之间的相似度:")
    res = word_vectors.similarity("坚强", "坚韧")
    print(res)
    
    res = word_vectors.most_similar(positive=["中国", "北京"], negative=["美国"], topn=5)
    for item in res:
        print(item[0], item[1])
    
    

2024-08-31 10:50:17,268 : INFO : loading projection weights from wiki300.model.bin
2024-08-31 10:50:22,794 : INFO : KeyedVectors lifecycle event {'msg': 'loaded (1035863, 100) matrix of type float32 from wiki300.model.bin', 'binary': True, 'encoding': 'utf8', 'datetime': '2024-08-31T10:50:22.794496', 'gensim': '4.3.3', 'python': '3.8.5 (default, Sep  4 2020, 02:22:02) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'load_word2vec_format'}


善良 0.8323723077774048
温柔 0.8308993577957153
自信 0.8281641602516174
内心 0.8140379190444946
纯真 0.8081669807434082
体贴 0.8071898221969604
渴望 0.8042596578598022
敢作敢为 0.8042051792144775
固执 0.8028025031089783
乐观 0.7942801117897034
坚强、坚韧之间的相似度:
0.612568
上海 0.6667948961257935
沈阳 0.6596426963806152
天津 0.6530011892318726
深圳 0.6351353526115417
石家庄 0.6298527717590332
